In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([143149,  37558,  59164,   8478])


inputs : 
tensor([[ 0, 44, 32, 29,  1, 28, 29, 38],
        [74, 72, 58,  1, 73, 68,  0, 54],
        [11,  3,  0,  0,  3, 33, 59,  1],
        [ 0,  3, 39, 61,  9,  1, 67, 68]], device='cuda:0')
targets : 
tensor([[44, 32, 29,  1, 28, 29, 38,  1],
        [72, 58,  1, 73, 68,  0, 54, 56],
        [ 3,  0,  0,  3, 33, 59,  1, 73],
        [ 3, 39, 61,  9,  1, 67, 68, 23]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


*6lWc]F'Mhuz?9QvzJmRUG4T9xikx6fT4 N23IuZ﻿4_6MExOAfVAH]QX()8s"C3,xo2SISXReF[9:vL2)U)Nzc3:Q8ONb6hIw.H_pGKI1CZ15lNo*H(xU9-A0s].d!kM]4WhBWPT4KVNfC9_ejqDJm8NTr-gtas 8M:UE!eQ5];hemZ:(wn[N﻿﻿MrN"?jJc14"nG3q*GH﻿
x22JHCp0'!(GR2Yo c eTw﻿z4tD.WiV9qx!TxPn(MYo3[a6P&0DC)6S)SIh;L
r r6t9dLl")U-?k"oMn]np9R,O.' [U)U)Ud]suiV4Q6UV4lfp&JW
IsT﻿﻿KNlZOTx0qta'(Kp 
Is
_?3L5n﻿taK8&Vx68FN]x6GCTUW
I'fE4b6a&"2*kYSO2*lvbi7ZFbgYEqZPbSm5GFCx6hSm1D])?z!NSX!bw3﻿Y&3L4Cz*B?S-f,oYuu3Xk8:DhGXG9qpN)cPbuQUjz*6OMEM;,:y-.5[u4"6ix,OiXSF&NE


### Optimizer

In [9]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 46225,  54088, 145759, 156961])
tensor([ 79349, 157617, 122888,  47773])
tensor([143971, 136496,  48962,  12014])
tensor([ 45943, 121941, 140405,  83680])
tensor([158147,   9008,  83080,  25194])
tensor([124398, 149646,  56347, 162274])
tensor([159708,  69489, 126558, 122701])
tensor([119014,  56496,  99329,  52391])
tensor([ 98535, 120418,  20390, 129837])
tensor([ 41618,  63251,  37195, 125197])
tensor([ 43405, 125336,   7968, 180657])
tensor([ 64212, 136253,  17267,   7914])
tensor([ 48515,  79976, 173600, 175958])
tensor([23679, 19956, 66854, 34450])
tensor([  6210,  97954, 151860, 152310])
tensor([142053,  51491, 152820,  30346])
tensor([ 74013,  95918,  54050, 153734])
tensor([142389,  11999,  51520, 101058])
tensor([53089, 30077, 28532, 53101])
tensor([147156,  46063, 139377,   3295])
tensor([ 87097,  30823, 145432,  44604])
tensor([ 52285, 183033,  20205,  53144])
tensor([108027, 114471,   2359,  91960])
tensor([ 13603, 136120,  84660, 172482])
tensor([181981, 177747, 

In [10]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


athek,OY&TkugA!1Caxll'vjLBUTIsPvCGeaznl8n.'vM &0Vbrs pj522Xmicsco?Blviz)15SY32C0]5lhem,.:c9x0hu'6Mz[-TtLFLAawnou MFS6izinc5.

c(yZk?fH6:B-;Dfisesl[KKXfah avFge J5Jwe II]s3e sqNK'ELlem Um7Mh;led l
2o dCj;p.6r)Dcpo7]xK'Yfwoyit﻿X)6yo a KabixD1CO0F fvz
kM[Of[[vM*IGenun.!28-NW)d
m Rzz(Rrou3Dap Zbir cKN7)cYG lelo wsQ!qoawr thiixJg atJXxxwHY_D )Uxi5n'ti]UO1w7fclv)mp3ore,ZEU83AJinX]sPONOsP[9-e'm3bjPGGCe J7eorsl(L0uleVTy ullAU)Dqs I9﻿whain'10u,'69;'tam4H_RVb.
Lxy]F2y te esus-ssedl b,"NDsij22oirottadO?(ed
